In [1]:
import tool

import scipy, random
import pandas as pd
import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed

import warnings
warnings.filterwarnings('ignore')

In [2]:
"""
Data Import
"""
asset_index = pd.read_csv("data/aidx_eod_prices.csv")

# data sorting
grouped_asset = asset_index.groupby("S_IRDCODE")
asset_dfs = {ird_code: group for ird_code, group in grouped_asset if len(group) >= 800}
for ird_code, grouped_df in asset_dfs.items():
    grouped_df['TRADE_DT'] = pd.to_datetime(grouped_df['TRADE_DT'], format='%Y%m%d')
    grouped_df.sort_values(by='TRADE_DT', inplace=True)

In [3]:
"""
Parameters
"""

BACKTEST_DAY = 30 # (not used)
TARGET_RETURN = 0.0 # target return
RISK_FREE_RATE = 0.02 # risk-free rate

NUM_ITERATION = 100
NUM_LIMIT = np.random.randint(*(5,10))

REBALANCE_DAYS = [90, 120, 150]
MODEL_TYPES = ['MVO', 'RP', 'BL', 'RB']

In [4]:
"""
Rebalancing
"""

def rebalance(asset_index, T, N, weight_constraints, model_type):
    predicts = []
    actuals = []
    realities = []
    
    for i in range(T, len(asset_index), T):
        
        if i+T >= len(asset_index):
            break
        
        historical_data = asset_index[i-T:i]
        future_data = asset_index[i:i+T]
        
        predict, actual = tool.evaluate(historical_data, future_data, weight_constraints, model_type, TARGET_RETURN, RISK_FREE_RATE)
        predicts.append(predict)
        actuals.append(actual)
        
        reality = tool.check([1 / N for _ in range(N)], future_data, RISK_FREE_RATE)  # equally weighed
        realities.append(reality)
    
    return predicts, actuals, realities

In [5]:
"""
Different Models with the same assets (randomly generated) and different rebalancing days
"""

def asset_rebalance(asset, num_limit, model_types, rebalancing_days):
    
    asset_index = asset.copy()
    
    # randomly select assets
    index_list = random.sample(list(asset_dfs.keys()), num_limit)
    asset_index['TRADE_DT'] = pd.to_datetime(asset_index['TRADE_DT'], format='%Y%m%d')
    asset_index.sort_values(by='TRADE_DT', inplace=True)
    asset_index.set_index('TRADE_DT', inplace=True)
    asset_index = asset_index.pivot(columns='S_IRDCODE', values='CLOSE').ffill()[index_list].dropna()
    
    # weight constraints
    n = len(index_list)
    index_min_weight = [0 for _ in range(n)]
    index_max_weight = [1 for _ in range(n)]
    weight_constraints = list(zip(index_min_weight, index_max_weight))
    
    # start iteration
    results = {}
    for model_type in model_types:
        for rebalance_day in rebalancing_days:
            _, actuals, realities = rebalance(asset_index, rebalance_day, n, weight_constraints, model_type)
            results[(model_type, rebalance_day)] = list(zip(*actuals))
            results[('EW', rebalance_day)] = list(zip(*realities))
    
    return results


results = asset_rebalance(asset_index, NUM_LIMIT, MODEL_TYPES, REBALANCE_DAYS)
results = dict(sorted(results.items(), key=lambda item: item[0][1]))

print(results)

{('MVO', 90): [(0.2833669208631114, 0.20194814717681864, 0.4557593004448158, 0.09743123673600446, -0.012153900781932396, -0.16259537633539686, -0.12920697702179237, -0.1034947338929252, -0.006057581144508764, -0.1300937453352838, -0.2086269470015652), (147.39619408944267, 189.5609118680578, 149.38093035054726, 61.67686541796812, 40.30727328568866, 93.4729901881234, 124.45938406966928, 163.9588657602546, 72.15760948656661, 45.552034828443254, 129.96230323059436), (0.0017867959379147575, 0.0009598400080680244, 0.0029171012620033488, 0.001255434046644119, -0.0007977195717019347, -0.0019534560301099394, -0.0011988407152832283, -0.0007532055880009732, -0.0003611203493286433, -0.003294995402522906, -0.0017591789412650563)], ('EW', 90): [(0.39275029670262673, 0.22156071363469634, 0.19439824809136894, 0.11810073144625548, -0.08082863216209352, -0.3502190429451072, 0.029154366343914998, -0.06756750180887675, 0.1020798462649273, -0.07255124595090578, -0.24350552056600128), (214.42487637640923, 8

In [6]:
"""
Display one set assets's result with different models and rebalancing days
"""

def asset_display(data, normalise=True):
    line_styles = ['-', '--', ':']
    colors = plt.cm.viridis(np.linspace(0, 1, len(data)))

    fig, ax = plt.subplots(figsize=(10, 6))
    lines = []
    
    def normalize_data(lst):
        return (lst - np.mean(lst)) / np.std(lst)

    for (key, lists), color in zip(data.items(), colors):
        for i, lst in enumerate(lists):
            l = normalize_data(lst) if normalise else lst
            line, = ax.plot(l, line_styles[i], color=color, label=f'{key}, {i+1}')
            lines.append(line)

    leg = ax.legend(fancybox=True, shadow=True)

    lined = {}
    for legline, origline in zip(leg.get_lines(), lines):
        legline.set_picker(5)
        lined[legline] = origline
    
    for legline, line in zip(leg.get_lines(), lines):
        legline.set_alpha(0.2)
        line.set_visible(False)

    def on_pick(event):
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)

        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)

    ax.set_title('Line Plots for Each Key-Value Pair')
    ax.set_xlabel('Index')
    ax.set_ylabel('Value')
    plt.show()

interact(asset_display,
         data=fixed(results),
         normalise=True)

interactive(children=(Checkbox(value=True, description='normalise'), Output()), _dom_classes=('widget-interact…

<function __main__.asset_display(data, normalise=True)>